## **Preprocess the Web Scrapped Data and Combine with PM2.5 Data**

In [1]:
import pandas as pd 
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt 
import seaborn as sns 
from bs4 import BeautifulSoup
import os 
import sys
import requests
import csv

In [2]:
file = open('Data_blr_tutiempo/Html_data/2018/1.html', 'rb').read()
texts = BeautifulSoup(file, 'lxml')

In [3]:
# print(texts.prettify())

In [4]:
headers = [header.text for header in texts.find_all('th')]
headers.pop(15)
headers.pop(14)
headers.pop(13)
headers.pop(12)
headers.pop(11)
headers.pop(10)
headers.pop(6)
headers.pop(0)
headers.pop(3)
headers.append('PM2.5')

headers

['T', 'TM', 'Tm', 'H', 'VV', 'V', 'VM', 'PM2.5']

In [5]:
# a = list()
# for tables in texts.findAll('table', {'class': "medias mensuales numspan"}):
#     for rows in tables.find_all('tr'):
#         cols = rows.find_all('td')
#         # Day = rows[0].text
#         if len(cols)!=0:
#             day = cols[0].text
#             T = cols[1].text
#             TM = cols[1].text
#             Tm = cols[1].text
#             SLP = cols[1].text
#             H = cols[1].text
#             PP = cols[1].text
#             VV = cols[1].text
#             V= cols[1].text
#             VM= cols[1].text
#             VG= cols[1].text
#             RA= cols[1].text
#             SN= cols[1].text
#             TS= cols[1].text
#             FG= cols[1].text
#             a.append([col.text for col in cols])
# pd.DataFrame(a)

**Upon inspection, it was noticed that several table entries (corresponding to the flag `td`) were not getting parsed, which resulted in some data loss. The data can be viewed on website [(Link)](https://en.tutiempo.net/climate/01-2018/ws-432950.html) but on inspecting the webpage (F12) the table entries are encrypted using the `span` html tag. This encryption pattern is same for all entries in month.
Since we are collecting data from 2018 - 2023 (6 years => 72 months), there could be 72 different encryption patterns. 
Hence, to avoid complexity, I have removed the table entries which cannot be parsed.
One example of the encryption is given using the below code.**

In [6]:
span_dictionaries = {1: 
                       {
                           'nttu':'2',
                           'ntlm':'1',
                           'ntyc':'.',
                           'nthj':'6',
                           'ntfs':'7',
                           'ntzb':'5',
                           'ntas':'8',
                           'nttn':'9',
                           'ntbb':'4',
                           'ntcd':'3',
                           'ntde':'0',
                           'ntio':'-',
                           '&nbsp': " "
                       }, 
                   2: {
                            'ntrs': '2',
                            'ntza': '3',
                            'ntvr': '.',
                            'ntbc': '0',
                            'ntqr': '8', 
                            'ntjk': '1', 
                            'ntzz': '-', 
                            'ntgy': '6', 
                            'ntbz': '5', 
                            'ntox': '7', 
                            'ntnt': '9', 
                            'ntaa': '4'
                        }, 
                   3: {
                            'ntpq': '2', 
                            'ntxo': '7', 
                            'ntux': '.', 
                            'ntgg': '4', 
                            'ntxy': '3', 
                            'nthi': '1',
                            'ntre': '8', 
                            'ntyy': '6', 
                            'ntll': '9', 
                            'ntaz': '5',
                            'ntab': '0', 
                            'ntkf': '-'
                       }, 
                   4: {
                            'ntvw': '2', 
                            'ntef': '3', 
                            'ntkk': '.', 
                            'ntno': '1', 
                            'ntjj': '-', 
                            'ntdr': '6', 
                            'nthy': '8', 
                            'ntpo': '7', 
                            'ntjg': '9', 
                            'ntgo': '5', 
                            'ntee': '4', 
                            'ntfg': '0'
                   }
                  }

# span_mapping_date = {
#     1: ["2018_1", "2018_2" "2018_3", "2018_12", "2019_1", "2019_8", "2019_9", "2020_1", "2020_4", "2020_6", "2020_7", "2020_9", "2020_10", "2021_2", "2021_7", "2021_8", "2021_12", "2022_1", "2022_4", "2022_11", "2023_4"],
#     2: ["2018_5", "2018_6", "2018_7", "2018_10", "2018_11", "2019_2", "2019_4", "2019_5", "2019_12", "2020_3", "2020_5", "2021_1", "2021_3", "2021_4", "2021_5", "2021_9", "2021_10", "2021_11", "2022_2", "2022_7", "2022_8", "2022_9", "2023_3", "2023_6"],
#     3: ["2018_4", "2018_8", "2019_10", "2019_7", "2020_2", "2021_5", "2022_6", "2022_10", "2022_12", "2023_8", "2023_9", "2023_10"],
#     4: ["2019_3", "2019_6", "2019_11", "2020_8", "2020_11", "2020_12", "2022_3", "2022_5", "2023_1", "2023_2", "2023_5", "2023_7", "2023_11", "2023_12"]
# }

# def find_key_by_value(value, dictionary = span_mapping_date):
#     for key, val in dictionary.items():
#         if value in val:
#             return key

def find_encoding(sample_encodings, dictionary = span_dictionaries): # sample_encodings = ['ntrs']
    for keys, vals in dictionary.items():            
        if sample_encodings[0] in list(vals.keys()):
            return keys

In [7]:
# span_dictionaries[find_key_by_value("2018_7")]['ntrs']
find_encoding(['ntrs'])

2

In [8]:
def met_data(month, year):
    file = open(f'Data_blr_tutiempo/Html_data/{year}/{month}.html', 'rb').read()
    texts = BeautifulSoup(file, 'lxml')

    # The encodings change everytime we scrap data from the website so we have to select the encoding before decrypting the data
    for rows in texts.find('table').find_all('tr'):
        cols = rows.find_all('td')
        col = list()
        if len(cols)!=0:
            for item in cols:
                if len(item.text)==0:
                    spans = [a.get('class')[0].strip() for a in item.find_all('span')]
                    # print(spans)
                    span_dictionary = span_dictionaries[find_encoding(spans)]
                    break

    data = list()
    finalD = list()
    
    # span_encoding = str(year)+"_"+str(month)
    # span_dictionary = span_dictionaries[find_key_by_value(span_encoding)]
    
    for rows in texts.find('table').find_all('tr'):
        cols = rows.find_all('td')
        col = list()
        if len(cols)!=0:
            for item in cols:
                if len(item.text)!=0:
                    col.append(item.text)
    
                else:
                    col.append("".join([span_dictionary[a.get('class')[0].strip()] for a in item.find_all('span')]))

            finalD.append(col)

    length = len(finalD)

    finalD.pop(length-1)
        
    for a in range(len(finalD)):
        finalD[a].pop(14)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(6)
        finalD[a].pop(4)
        finalD[a].pop(0)
        
    return finalD

In [9]:
met_data(1, 2018)

[['22.2', '28.4', '16.6', '53', '7.7', '0.9', '5.4'],
 ['22.1', '28.9', '17.8', '58', '5.1', '1.1', '-'],
 ['22.2', '29', '18.4', '62', '6', '3.9', '5.4'],
 ['21.6', '27.6', '17', '61', '6.3', '5', '7.6'],
 ['20.7', '28.3', '15.5', '54', '6.3', '4.8', '7.6'],
 ['20.2', '27.2', '14.7', '52', '5.8', '2.8', '5.4'],
 ['20.4', '27.2', '15.2', '52', '6.3', '4.1', '7.6'],
 ['20.1', '27.2', '14.8', '50', '6.9', '6.5', '7.6'],
 ['21.2', '26.6', '15.5', '59', '6.3', '6.7', '9.4'],
 ['21.8', '27.1', '18.2', '66', '5.8', '5.7', '11.1'],
 ['22.7', '28.3', '17.3', '65', '6.9', '4.8', '9.4'],
 ['23.1', '29.4', '18.7', '64', '6.3', '3.9', '7.6'],
 ['23.4', '30.6', '17.6', '48', '6.3', '4.1', '5.4'],
 ['23.3', '31', '16.3', '42', '5.8', '2.8', '11.1'],
 ['22.4', '30.7', '16.3', '49', '6.3', '3.3', '7.6'],
 ['22.9', '30.5', '17.1', '49', '6.3', '3.9', '7.6'],
 ['21.8', '29.9', '16.8', '49', '5.5', '5.6', '7.6'],
 ['21.2', '28.6', '15.3', '41', '6.3', '6.7', '11.1'],
 ['20.7', '28.7', '14.3', '48', '6.9'

In [17]:
# To check if the encodings we have selected are correctly decoding the data or not
for year in range(2018, 2024):
    for month in range(1, 13):
        try:
            met_data(month, year)
        except:
            print(month, year)
        # else:
        #     met_data(month, year)

In [11]:
# pd.DataFrame(met_data(1, 2018))

In [12]:
# def met_data(month, year):
#     '''
#     This function parses the html data (atmospheric data) collected previously and also removed features/columns which contains 
#     null values ('-', " ") based on the year and month provided.
#     '''
#     file_path = f'Data_blr_tutiempo/Html_data/{year}/{month}.html'
#     file_html = open(file_path, 'rb')
#     plain_text = file_html.read()

#     tempD = list()
#     finalD = list()

#     soup = BeautifulSoup(plain_text, 'lxml')

#     for table in soup.find_all('table', {'class': 'medias mensuales numspan'}):       
#         rows = table.find_all('tr')        
        
#         for row in rows:
#             cols = row.find_all('td')
#             if len(cols)!=0:
#                 finalD.append([a.text for a in cols]) 

#     length = len(finalD)

#     finalD.pop(length-1)
    
#     for a in range(len(finalD)):
#         finalD[a].pop(6) # PP
#         finalD[a].pop(13) # 
#         finalD[a].pop(12)
#         finalD[a].pop(11)
#         finalD[a].pop(10)
#         finalD[a].pop(9)
#         finalD[a].pop(0)
#         finalD[a].pop(3) # SLP
    
#     return finalD

In [13]:
met_data(7, 2018)

[['24.2', '29.5', '20.6', '75', '6.9', '9.4', '14.8'],
 ['24.8', '29.4', '20.9', '68', '6.9', '10.9', '14.8'],
 ['24.4', '30.2', '20.7', '68', '6.9', '8.1', '11.1'],
 ['24.1', '29', '20.5', '67', '6.9', '11.1', '18.3'],
 ['24.5', '29.6', '19.7', '65', '6.9', '9.3', '14.8'],
 ['22.8', '29.6', '20', '80', '6.9', '10.4', '18.3'],
 ['22.9', '26.3', '20.5', '82', '6.9', '11.3', '16.5'],
 ['22.4', '26.4', '20.3', '86', '8', '6.7', '9.4'],
 ['22.8', '26.4', '20.7', '85', '6.9', '8.7', '14.8'],
 ['22.7', '26.9', '20.6', '86', '7.4', '9.3', '13'],
 ['22.5', '27', '20.3', '87', '6.3', '9.3', '14.8'],
 ['22.8', '27.3', '20.1', '80', '6.9', '11.7', '14.8'],
 ['22.2', '27.4', '20.5', '86', '7.7', '10.9', '14.8'],
 ['22.4', '26.5', '20.3', '86', '6.9', '8.5', '11.1'],
 ['23.6', '27.2', '20.4', '74', '6.9', '10', '13'],
 ['22.2', '28.4', '20.2', '79', '6.9', '13.9', '18.3'],
 ['22.4', '26.2', '19.8', '79', '6.9', '12.2', '18.3'],
 ['23.8', '28', '19.7', '74', '6.9', '12.8', '18.3'],
 ['23', '28.6', '

> Below code is used to create the final dataset where we merge the AQI data (dependent variable) and atmospheric data (independent variables; web scraped html data)

In [14]:
start_year = 2018
end_year = 2023
if not os.path.exists('Final_dataset/Final_data'):
    os.makedirs('Final_dataset/Final_data')
    
for year in range(start_year, end_year+1):
    final_data = list()
    
    with open(f'Final_dataset/Final_data/{year}.csv', 'w', encoding='utf-8') as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(headers)

    with open(f'Data_blr_AQI/{year}.csv', 'r', encoding='utf-8') as aqi:
        aqi_data = csv.reader(aqi)
        content = [row[1] for row in aqi_data]
        content.pop(0)
    
    for month in range(1, 13):
        print(year, month)
        temp = met_data(month, year)
        final_data += temp

    # Merge the AQI and web scraped data
    for day in range(0, 365):
        final_data[day].insert(8, content[day])
        # print(final_data[day])

    
    with open(f'Final_dataset/Final_data/{year}.csv', 'a', encoding='utf-8') as fp:
        wr = csv.writer(fp, dialect='excel')
        for row in final_data:
            flag=0
            for elem in row:
                if elem=="" or elem=="-":
                    flag=1 # Many rows contains empty values; Inorder to remove them
            if flag!=1:
                wr.writerow(row)

2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
2021 1
2021 2
2021 3
2021 4
2021 5
2021 6
2021 7
2021 8
2021 9
2021 10
2021 11
2021 12
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11
2023 12


In [15]:
def data_combine(year, cs): # cs = chunnksize
    '''
    This function extracts the content of the csv file containing the final data year-wise
    cs = chunksize; if RAM is a constraint and there is lots of data in the csv files
    '''
    for a in pd.read_csv(f'Final_dataset/Final_data/{year}.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist

In [16]:
combined_data = list()
for year in range(start_year, end_year+1):
    combined_data.extend(data_combine(year, 200))

with open('Final_dataset.csv', 'w') as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow(headers)
    wr.writerows(combined_data)